In [1]:
import open3d as o3d
import os
import numpy as np

this_path = os.getcwd()
data_path = os.path.abspath(os.path.join(this_path, '..', '..', 'data'))
obj_file = os.path.join(data_path, 'gralha_azul_torre.obj')

mesh = o3d.io.read_triangle_mesh(obj_file)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def get_mesh_min_max(mesh):
  """
  Calculates the minimum and maximum x, y, and z coordinates of a mesh.

  Args:
    mesh: The Open3D mesh object.

  Returns:
    A dictionary containing the minimum and maximum x, y, and z values.
  """

  # Get the mesh vertices as a NumPy array
  vertices = np.asarray(mesh.vertices)

  # Calculate the minimum and maximum values along each axis (x, y, z)
  min_x, min_y, min_z = np.min(vertices, axis=0)
  max_x, max_y, max_z = np.max(vertices, axis=0)

  # Return the results in a dictionary
  return {
      "min_x": min_x,
      "min_y": min_y,
      "min_z": min_z,
      "max_x": max_x,
      "max_y": max_y,
      "max_z": max_z,
  }

In [3]:
def is_point_inside_mesh(mesh, point):
    """
    Checks if a point is inside a mesh.

    Args:
      mesh: The Open3D mesh object.
      point: A tuple (x, y, z) representing the point's coordinates.

    Returns:
      True if the point is inside the mesh, False otherwise.
    """

    # Create a RaycastingScene with the mesh
    scene = o3d.t.geometry.RaycastingScene()
    scene.add_triangles(o3d.t.geometry.TriangleMesh.from_legacy(mesh)) 

    # Define the ray origin (your point) and direction (any direction will do)
    ray_origin = point
    ray_direction = [1.0, 0.0, 0.0]  # Example direction: along the positive x-axis

    # Perform the ray cast
    ans = scene.cast_rays(o3d.core.Tensor([ray_origin], dtype=o3d.core.Dtype.Float32))

    # Extract the hit information
    hit = ans['t_hit'].numpy() 

    # If there's a hit (not NaN) and the number of hits is odd, the point is inside
    return not np.isnan(hit[0]) and int(hit[0]) % 2 == 1

In [13]:
def filter_points_by_mesh(pcd, mesh):
    """
    Filters a point cloud to keep only the points inside a mesh.

    Args:
      pcd: The Open3D point cloud object.
      mesh: The Open3D mesh object.

    Returns:
      An Open3D point cloud containing only the points inside the mesh.
    """

    # Create a RaycastingScene with the mesh
    scene = o3d.t.geometry.RaycastingScene()
    scene.add_triangles(o3d.t.geometry.TriangleMesh.from_legacy(mesh))

    # Get the point cloud points as a NumPy array
    points = np.asarray(pcd.points)

    # Prepare an array to store the ray origins (the point cloud points)
    ray_origins = o3d.core.Tensor(points, dtype=o3d.core.Dtype.Float32)

    # Choose an arbitrary ray direction (any direction will work)
    ray_direction = [1.0, 0.0, 0.0] 

    # Perform ray casting for all points in the point cloud
    ans = scene.cast_rays(ray_origins)

    # Extract hit information
    hit = ans['t_hit'].numpy()

    # Create a mask to select points with an odd number of hits
    inside_mask = (np.isfinite(hit)) & (hit.astype(int) % 2 == 1)

    # Filter the point cloud using the mask
    filtered_points = o3d.geometry.PointCloud()
    filtered_points.points = o3d.utility.Vector3dVector(points[inside_mask])

    return filtered_points

In [4]:
import numpy as np

vertices = np.asarray(mesh.vertices)
vertices

array([[ 1.33616102, 50.75621796, -1.05936503],
       [ 1.33881903, 50.76016998, -1.056615  ],
       [ 1.33728504, 50.75788879, -1.05735195],
       ...,
       [-5.86057281, 15.71444893, -3.02037001],
       [-5.85783005, 15.7174778 , -3.02104092],
       [-5.85627174, 15.71181488, -3.014678  ]])

In [5]:
aabb = mesh.get_axis_aligned_bounding_box()
aabb.color = (1, 0, 0)
obb = mesh.get_oriented_bounding_box()
obb.color = (0, 1, 0)
# o3d.visualization.draw_geometries([mesh, aabb, obb])

In [6]:
aabb

AxisAlignedBoundingBox: min: (-12.4798, 8.31601, -5.35876), max: (12.4798, 66.95, 5.35876)

In [7]:
obb

OrientedBoundingBox: center: (-0.0752666, 37.5985, 0.24228), extent: 58.7123, 24.9598, 10.7357)

In [8]:
get_mesh_min_max(mesh)

{'min_x': -12.479758262634277,
 'min_y': 8.31600570678711,
 'min_z': -5.358763217926025,
 'max_x': 12.479758262634277,
 'max_y': 66.95001983642578,
 'max_z': 5.358759880065918}

In [9]:
type(aabb)

open3d.cpu.pybind.geometry.AxisAlignedBoundingBox

In [10]:
aabb.get_box_points()

std::vector<Eigen::Vector3d> with 8 elements.
Use numpy.asarray() to access data.

In [12]:
np.asarray(aabb.get_box_points())

array([[-12.47975826,   8.31600571,  -5.35876322],
       [ 12.47975826,   8.31600571,  -5.35876322],
       [-12.47975826,  66.95001984,  -5.35876322],
       [-12.47975826,   8.31600571,   5.35875988],
       [ 12.47975826,  66.95001984,   5.35875988],
       [-12.47975826,  66.95001984,   5.35875988],
       [ 12.47975826,   8.31600571,   5.35875988],
       [ 12.47975826,  66.95001984,  -5.35876322]])